In [12]:
import os
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
import os, sys

# 뉴스심리지수(NSI, News Sentiment Index)

직전 7일간 뉴스기사에 나타난 경제심리를 일 단위로 지수화한 것임

인터넷 포털사이트의 경제분야 뉴스기사(한국지능정보사회진흥원 제공)에서 표본문장을 무작위로 추출하여 각 문장에 나타난 경제심리를 기계학습(machine learning)을 통해 긍정, 부정, 중립으로 분류하고 다음과 같은 산식에 따라 지수를 산출. 자세한 내용은 "뉴스심리지수(NSI) 개요 및 특성" 참고

NSI = (기간내 긍정문장 수 − 부정문장 수)/(기간내 긍정문장 수 + 부정문장 수) × 100 + 100

본 지표는 한국은행 경제통계국에서 개발하여 경제심리 변화의 신속한 파악에 도움을 주고자 조사연구 결과물의 형태로 시험공개하며, 국가승인통계가 아니므로 인용과 활용에 주의를 요함

http://ecos.bok.or.kr/EIndex.jsp

## 01. Train Data Load

In [13]:
train_df = pd.read_csv('./result/train_data(new_past_d,new_stk_data, new_external_data).csv')
test_df = pd.read_csv('./result/test_data(new_past_d,new_stk_data, new_external_data).csv')

In [3]:
train_df

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,past_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,...,aet_amt_change,open_d,volume_d,change_d,mean_volume,mean_open,mean_high,mean_low,mean_close,mean_abs_change
0,0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A002270,20190321,350,120.750000,2,6,4,2,...,923285.0,10600.0,2654683.0,-0.009434,3.057820e+03,587925.989405,595956.222494,579696.013040,595411.790546,0.013545
1,1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A005070,20190327,19,23.250923,2,6,4,2,...,152500.0,783.0,9399085.0,0.002558,6.215593e+05,9806.215159,10083.665037,9533.344743,10003.691117,0.026670
2,2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A005930,20190320,1,58.914160,2,6,4,2,...,440500.0,27400.0,1848.0,0.000000,8.282941e+06,45025.487368,45494.950285,44570.081500,45170.081500,0.012765
3,3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A033270,20190326,75,21.489474,2,6,4,2,...,152450.0,4415.0,145717.0,-0.022676,2.900570e+05,24804.080685,25477.752241,24192.547677,24917.685412,0.021596
4,4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A036000,20190611,133,41.692308,2,6,4,2,...,301566.0,48350.0,1292799.0,-0.001035,2.677373e+05,5925.521597,6054.246129,5792.220864,5912.465363,0.021279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,681467,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A253450,20200130,130,32.022831,2,3,2,1,...,84700.0,13050.0,25117828.0,0.298450,2.364029e+05,85036.614173,86614.041995,83327.690289,84882.808399,0.020508
681468,681468,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A264450,20201217,8,23.487805,2,3,2,1,...,52900.0,3745.0,99859.0,-0.019920,6.442973e+04,15114.947826,15477.109783,14766.867391,15119.502174,0.021951
681469,681469,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A267320,20201007,47,8.906250,2,3,2,1,...,81600.0,3525.0,63304.0,-0.035714,1.074191e+06,1879.232102,1920.221709,1846.302540,2143.690531,0.012190
681470,681470,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A293490,20200914,61,31.100000,2,3,2,1,...,87000.0,3785.0,351186.0,-0.001340,1.513158e+06,50502.000000,51475.333333,49558.000000,50268.666667,0.023183


In [19]:
test_df

,Unnamed: 0,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,...,aet_amt_change,open_d,volume_d,change_d,mean_volume,mean_open,mean_high,mean_low,mean_close,mean_abs_change
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,...,NaN,13300.0,1930122.0,-0.014981,1.859878e+06,13298.084760,13491.597392,13103.480033,13290.309698,0.014526
1,1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,...,1.336500e+06,65886.0,1783575.0,-0.015448,3.079929e+05,30852.931540,31516.180929,30064.927465,31274.595762,0.017717
2,2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,...,1.183550e+06,31600.0,241315.0,-0.015823,1.645605e+05,30529.807692,31236.675824,29781.730769,30427.060440,0.023442
3,3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,...,9.176117e+05,2100.0,6615258.0,0.091787,7.292597e+05,2935.981255,3011.268134,2858.189894,2932.819071,0.025209
4,4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,...,NaN,46350.0,523303.0,0.002157,1.513158e+06,50502.000000,51475.333333,49558.000000,50268.666667,0.023183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,...,3.368000e+05,NaN,NaN,NaN,9.611750e+04,256808.883456,259797.473513,254081.499593,256982.885086,0.012186
70592,70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,...,8.253750e+05,NaN,NaN,NaN,7.016480e+04,8223.781581,8282.868786,8147.864711,8222.860636,0.008042
70593,70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,...,NaN,NaN,NaN,NaN,8.560609e+05,1.770270,1.870270,1.405405,952.164865,0.004922
70594,70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,...,1.360840e+07,NaN,NaN,NaN,3.444963e+04,85423.716381,87124.286879,83760.146699,85415.077425,0.017916


## 02. NSI Data Load

In [5]:
nsi_df = pd.read_csv('../../external/result/nsi_data.csv')

In [6]:
nsi_df

,date,nsi
0,20151201,119.02
1,20151202,119.58
2,20151203,121.65
3,20151204,120.14
4,20151205,119.18
...,...,...
1853,20201227,119.92
1854,20201228,119.93
1855,20201229,118.98
1856,20201230,120.49


In [7]:
train_df = pd.merge(left=train_df, right=nsi_df, how = 'left', left_on='byn_dt',right_on='date')
train_df.drop(['date'],axis = 1)

,Unnamed: 0,act_id,iem_cd,byn_dt,hold_d,past_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,...,open_d,volume_d,change_d,mean_volume,mean_open,mean_high,mean_low,mean_close,mean_abs_change,nsi
0,0,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A002270,20190321,350,120.750000,2,6,4,2,...,10600.0,2654683.0,-0.009434,3.057820e+03,587925.989405,595956.222494,579696.013040,595411.790546,0.013545,110.22
1,1,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A005070,20190327,19,23.250923,2,6,4,2,...,783.0,9399085.0,0.002558,6.215593e+05,9806.215159,10083.665037,9533.344743,10003.691117,0.026670,111.06
2,2,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A005930,20190320,1,58.914160,2,6,4,2,...,27400.0,1848.0,0.000000,8.282941e+06,45025.487368,45494.950285,44570.081500,45170.081500,0.012765,108.19
3,3,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A033270,20190326,75,21.489474,2,6,4,2,...,4415.0,145717.0,-0.022676,2.900570e+05,24804.080685,25477.752241,24192.547677,24917.685412,0.021596,111.85
4,4,0000c7a5a5a1db9c7e890da984b20d544c09cb6f51c546...,A036000,20190611,133,41.692308,2,6,4,2,...,48350.0,1292799.0,-0.001035,2.677373e+05,5925.521597,6054.246129,5792.220864,5912.465363,0.021279,100.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,681467,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A253450,20200130,130,32.022831,2,3,2,1,...,13050.0,25117828.0,0.298450,2.364029e+05,85036.614173,86614.041995,83327.690289,84882.808399,0.020508,108.11
681468,681468,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A264450,20201217,8,23.487805,2,3,2,1,...,3745.0,99859.0,-0.019920,6.442973e+04,15114.947826,15477.109783,14766.867391,15119.502174,0.021951,119.96
681469,681469,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A267320,20201007,47,8.906250,2,3,2,1,...,3525.0,63304.0,-0.035714,1.074191e+06,1879.232102,1920.221709,1846.302540,2143.690531,0.012190,113.57
681470,681470,fff9cf3aaffd32dcbf70dce8e5216411f9d38537b477d6...,A293490,20200914,61,31.100000,2,3,2,1,...,3785.0,351186.0,-0.001340,1.513158e+06,50502.000000,51475.333333,49558.000000,50268.666667,0.023183,106.32


In [8]:
test_df = pd.merge(left=test_df, right=nsi_df, how = 'left', left_on='byn_dt',right_on='date')
test_df.drop(['date'],axis = 1)

,Unnamed: 0,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,...,open_d,volume_d,change_d,mean_volume,mean_open,mean_high,mean_low,mean_close,mean_abs_change,nsi
0,0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,...,13300.0,1930122.0,-0.014981,1.859878e+06,13298.084760,13491.597392,13103.480033,13290.309698,0.014526,106.54
1,1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,...,65886.0,1783575.0,-0.015448,3.079929e+05,30852.931540,31516.180929,30064.927465,31274.595762,0.017717,102.73
2,2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,...,31600.0,241315.0,-0.015823,1.645605e+05,30529.807692,31236.675824,29781.730769,30427.060440,0.023442,109.37
3,3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,...,2100.0,6615258.0,0.091787,7.292597e+05,2935.981255,3011.268134,2858.189894,2932.819071,0.025209,121.26
4,4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,...,46350.0,523303.0,0.002157,1.513158e+06,50502.000000,51475.333333,49558.000000,50268.666667,0.023183,119.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,...,NaN,NaN,NaN,9.611750e+04,256808.883456,259797.473513,254081.499593,256982.885086,0.012186,124.28
70592,70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,...,NaN,NaN,NaN,7.016480e+04,8223.781581,8282.868786,8147.864711,8222.860636,0.008042,121.99
70593,70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,...,NaN,NaN,NaN,8.560609e+05,1.770270,1.870270,1.405405,952.164865,0.004922,127.37
70594,70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,...,NaN,NaN,NaN,3.444963e+04,85423.716381,87124.286879,83760.146699,85415.077425,0.017916,122.60


In [9]:
test_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [10]:
test_df

,act_id,iem_cd,byn_dt,hist_d,submit_id,hold_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,...,volume_d,change_d,mean_volume,mean_open,mean_high,mean_low,mean_close,mean_abs_change,date,nsi
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A032640,20200522,153,IDX00001,0,1,9,3,2,...,1930122.0,-0.014981,1.859878e+06,13298.084760,13491.597392,13103.480033,13290.309698,0.014526,20200522,106.54
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A160600,20190823,335,IDX00002,0,1,9,3,2,...,1783575.0,-0.015448,3.079929e+05,30852.931540,31516.180929,30064.927465,31274.595762,0.017717,20190823,102.73
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A234340,20200611,139,IDX00003,0,1,9,3,2,...,241315.0,-0.015823,1.645605e+05,30529.807692,31236.675824,29781.730769,30427.060440,0.023442,20200611,109.37
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A131760,20200120,236,IDX00004,0,1,9,3,2,...,6615258.0,0.091787,7.292597e+05,2935.981255,3011.268134,2858.189894,2932.819071,0.025209,20200120,121.26
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A293490,20201217,9,IDX00005,0,1,9,3,2,...,523303.0,0.002157,1.513158e+06,50502.000000,51475.333333,49558.000000,50268.666667,0.023183,20201217,119.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70591,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A000810,20201127,23,IDX70592,0,1,6,0,5,...,NaN,NaN,9.611750e+04,256808.883456,259797.473513,254081.499593,256982.885086,0.012186,20201127,124.28
70592,564af3cf975ed4fa05148ada689d5002e9c2d43dee9481...,A005945,20200731,103,IDX70593,0,1,6,0,5,...,NaN,NaN,7.016480e+04,8223.781581,8282.868786,8147.864711,8222.860636,0.008042,20200731,121.99
70593,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A004740,20170704,858,IDX70594,0,1,4,4,2,...,NaN,NaN,8.560609e+05,1.770270,1.870270,1.405405,952.164865,0.004922,20170704,127.37
70594,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A001630,20201215,11,IDX70595,0,1,4,4,2,...,NaN,NaN,3.444963e+04,85423.716381,87124.286879,83760.146699,85415.077425,0.017916,20201215,122.60


In [11]:
train_df.to_csv('./result/train_data(new_past_d,new_stk_data, new_external_data, nsi).csv',index = False)
test_df.to_csv('./result/test_data(new_past_d,new_stk_data, new_external_data, nsi).csv',index = False)

KeyboardInterrupt: 